

# Chatbot untuk kaamala resort



In [15]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
import numpy as np

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
df = pd.read_excel('dataset.xlsx')
df.head()

,In,Class
0,tell me about kaamala resort,description
1,can you tell me what is kaamala resort,description
2,describe kaamala resort,description
3,what are the facilities at kaamala resort,facilities
4,what are the resort facilities,facilities


In [17]:
def clean_corpus(text):

    stop_words = stopwords.words('english')
    ps = PorterStemmer()
    nlp = spacy.load('en_core_web_sm')

    text = text.lower()
    text = re.findall(r'[a-z]+', text)

    text = [i for i in text if i not in stop_words]

    text = nlp(" ".join(text))
    text = [i.lemma_ for i in text]

    text = [ps.stem(i) for i in text]

    return " ".join(text)

In [18]:
df['In'] = df['In'].apply(clean_corpus)

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['In'])
sequences = tokenizer.texts_to_sequences(df['In'])

total_words = len(tokenizer.word_index) + 1
maxlen = max([len(x) for x in sequences])
sequences = pad_sequences(sequences, maxlen=maxlen)

In [21]:
classi = [i for i in df['Class'].drop_duplicates()]
encode = [i for i in range(len(classi))]

label_dict = dict(zip(classi, encode))
label = df['Class'].replace(label_dict)

In [22]:
from tensorflow.keras import layers, Model, Input, optimizers, callbacks

In [23]:
inputs = Input(shape=(maxlen,))
embedding = layers.Embedding(total_words, 100, input_length=maxlen)(inputs)
x = layers.Bidirectional(layers.LSTM(128))(embedding)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(len(classi), activation='softmax')(x)

model = Model(inputs=inputs, outputs=x)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [24]:
class myCallback(callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.99:
            self.model.stop_training =True

In [25]:
callback = myCallback()
history = model.fit(sequences, label,
                    epochs=250,
                    callbacks=callback)

Epoch 1/250
1/1 [==============================] - 4s 4s/step - loss: 1.7915 - accuracy: 0.1429
Epoch 2/250
1/1 [==============================] - 0s 12ms/step - loss: 1.7900 - accuracy: 0.2857
Epoch 3/250
1/1 [==============================] - 0s 12ms/step - loss: 1.7886 - accuracy: 0.2857
Epoch 4/250
1/1 [==============================] - 0s 14ms/step - loss: 1.7872 - accuracy: 0.3333
Epoch 5/250
1/1 [==============================] - 0s 14ms/step - loss: 1.7858 - accuracy: 0.4286
Epoch 6/250
1/1 [==============================] - 0s 13ms/step - loss: 1.7845 - accuracy: 0.4286
Epoch 7/250
1/1 [==============================] - 0s 17ms/step - loss: 1.7831 - accuracy: 0.4286
Epoch 8/250
1/1 [==============================] - 0s 16ms/step - loss: 1.7818 - accuracy: 0.4762
Epoch 9/250
1/1 [==============================] - 0s 16ms/step - loss: 1.7805 - accuracy: 0.5714
Epoch 10/250
1/1 [==============================] - 0s 15ms/step - loss: 1.7791 - accuracy: 0.6667
Epoch 11/250
1/1 [===

In [26]:
model.save('chatbot_kaamala.h5')

In [27]:
text = 'what are the facilities at kaamala resort'
text = clean_corpus(text)
text = tokenizer.texts_to_sequences([text])
text = np.array(text).reshape(-1)
text = pad_sequences([text], maxlen=maxlen)

predict = model.predict(text)
predict = predict.argmax()

predict

1/1 [==============================] - 1s 666ms/step


1

In [32]:
from pickle import dump, load

In [37]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))
tok = load(open('tokenizer.pkl', 'rb'))

In [40]:
maxlen

5